# Data Analysis Jupyter Notebook

This Jupyter Notebook is a template for data analysis. It includes installation of necessary libraries, loading datasets, and displaying basic information and statistics for each dataset.


## Environment preparation

First of all, we will install all the required packages:


In [ ]:
#capture
#!python -m pip install --upgrade pip
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn
#!pip install scipy
#!pip install geopandas

## Data Understanding and preparation

We will read the datasets in order to understand the information that they contain, and adapt them to the desired format.

In [ ]:
# Import necessary libraries
import pandas as pd

# Load the three datasets
incidents_dataset = pd.read_csv('./project_datasets/incidents.csv', low_memory=False)
pbsy_dataset = pd.read_csv('./project_datasets/povertyByStateYear.csv')
ysdh_dataset = pd.read_csv('./project_datasets/year_state_district_house.csv')

After reading the datasets, we will execute some commands to have a preview about the information.

With `info()`, we will see the names of the columns of the datasets, the data type and the amount of non null values that they have. With `head()`, the first 5 rows of each dataset, that they will help us to have the idea of the dataset row types. Eventually, with `describe()` we will obtain the statisctical values of the numerical columns.

In [ ]:
# Display basic information about each dataset
print("\n------- Incidents Info:-------")
print(incidents_dataset.info())

print("\n------- Dataset 2 Info:-------")
print(pbsy_dataset.info())

print("\n------- Dataset 3 Info:-------")
print(ysdh_dataset.info())

In [ ]:
# Display the first few rows of each dataset
print("------- Incidents -------")
print(incidents_dataset.head())

print("\n------- Poverty By State and Year -------")
print(pbsy_dataset.head())

print("\n------- Year State District House -------")
print(ysdh_dataset.head())

Here we can see that the data types of some of the columns are not correct, so we will fix them. For example, `date` columns has data type `object`, so we will convert it on datetime and some columns that contain numeric values are as `object` type as well, so we will change them. Also here, the modifications are done in the incidents dataset.

In [ ]:
# Convert the 'date' column to datetime
incidents_dataset['date'] = pd.to_datetime(incidents_dataset['date'])

# Convert numeric data from object to numeric, handling errors
incidents_dataset['state_senate_district'] = pd.to_numeric(incidents_dataset['state_senate_district'], errors='coerce')
incidents_dataset['min_age_participants'] = pd.to_numeric(incidents_dataset['min_age_participants'], errors='coerce')
incidents_dataset['max_age_participants'] = pd.to_numeric(incidents_dataset['max_age_participants'], errors='coerce')
incidents_dataset['avg_age_participants'] = pd.to_numeric(incidents_dataset['avg_age_participants'], errors='coerce')
incidents_dataset['n_participants_child'] = pd.to_numeric(incidents_dataset['n_participants_child'], errors='coerce').astype('Int64')
incidents_dataset['n_participants_teen'] = pd.to_numeric(incidents_dataset['n_participants_teen'], errors='coerce').astype('Int64')
incidents_dataset['n_participants_adult'] = pd.to_numeric(incidents_dataset['n_participants_adult'], errors='coerce').astype('Int64')

# Verify the data types after conversion
print("\nData Types After Conversion:\n ---- Dataset 1 ----\n", incidents_dataset.dtypes)
print("\n ---- Dataset 2 ----\n", pbsy_dataset.dtypes)
print("\n ---- Dataset 3 ----\n", ysdh_dataset.dtypes)


In [ ]:
# Display basic statistics for each dataset
print("\n------- Incidents Statistics:-------")
print(incidents_dataset.describe())

print("\n------- Poverty By State and Year Statistics:-------")
print(pbsy_dataset.describe())

print("\n------- Dataset 3 Statistics:-------")
print(ysdh_dataset.describe())


### Datasets columns description

After analyzing the datasets with the previous commands, we will briefly describe each of the columns of the three datasets:

#### Incidents dataset (incidents.csv)

- **Date of Incident (`date`):** This variable represents the date when the gun incident occurred.
- **State (`state`):** Indicates the state where the incident took place.
- **City or County (`city_or_county`):** Specifies the city or county where the incident occurred.
- **Address (`address`):** Represents the specific address where the incident took place.
- **Geographical Coordinates (`latitude, longitude`):** Provides the latitude and longitude of the incident location.
- **Congressional District (`congressional_district`):** Specifies the congressional district where the incident occurred.
- **State House District (`state_house_district`):** Represents the state house district of the incident.
- **State Senate District (`state_senate_district`):** Indicates the state senate district where the incident took place.
- **Participant Age (`participant_age1`):** Represents the exact age of one randomly chosen participant in the incident.
- **Participant Age Group (`participant_age_group1`):** Specifies the age group of one randomly chosen participant.
- **Participant Gender (`participant_gender1`):** Indicates the gender of one randomly chosen participant.
- **Minimum, Average, and Maximum Age of Participants (`min_age_participants, avg_age_participants, max_age_participants`):** Provide statistical measures of participant ages.
- **Number of Participants by Age Group (`n_participants_child, n_participants_teen, n_participants_adult`):** Gives the count of participants in different age groups.
- **Number of Males and Females (`n_males, n_females`):** Specifies the count of male and female participants.
- **Number of People Killed and Injured (`n_killed, n_injured`):** Represents the count of people killed and injured in the incident.
- **Number of Arrested and Unharmed Participants (`n_arrested, n_unharmed`):** Indicates the count of participants arrested and unharmed.
- **Total Number of Participants (`n_participants`):** Represents the total number of participants in the incident.
- **Additional Notes (`notes`):** Provides additional information or notes about the incident.
- **Incident Characteristics (`incident_characteristics1, incident_characteristics2`):** Specifies the characteristics of the incident.

#### Poverty by state and year dataset (povertyByStateYear.csv)

- **State (`state`):** The name of the state.
- **Year (`year`):** The year for which the poverty rate is recorded.
- **Poverty percentage (`povertyPercentage`):** The percentage of the population in poverty for a specific state and year.

#### Year state district house dataset (year_state_district_house.csv)

- **Year (`year`):** The year of the election.
- **State (`state`):** The name of the state for which election results are recorded.
- **State (`congressional_district`):** The congressional district number for which election results are recorded.
- **Party (`party`):** The political party associated with the candidate.
- **Candidate votes (`candidatevotes`):** The number of votes received by a specific candidate in a particular congressional district.
- **Total votes (`totalvotes`):** The total number of votes cast in a particular congressional district.

### Data quality assessment

After having the idea of the content of the dataset, we will perform an analysis of the quality of the data.

Firstly, we will check for missing values in the dataset 1 columns. We will perform this analysis in the dataset of incidents only, because the other two do not have missing values.

With the next graph we can see the distribution of missing values for each column. For example, we can see that the column with more missing values is  `incident_characteristic2`. This occurs because most of the incidents do not have two characteristics, but just one, since we can see that `incident_characteristic1` has almost no missing values.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Check for missing values
missing_values = incidents_dataset.isnull().sum()
#print("\nMissing Values:\n", missing_values)

# Plotting the sum of missing values for each column using a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_values.index, y=missing_values.values, hue=missing_values.index, palette='flare', legend=False)
plt.xticks(rotation=45, ha='right')
plt.title('Sum of Missing Values in Incidents Dataset')
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.show()

Cases that do not have the characteristic specified (`incident_characteristics1` = *null*), would be removed, since we would not know what was the incident about, and it does not make sense to analyze them.

The data that doesn't have any `address` and any `latitude` and `longitude` will be eliminated since it does not make sense that an incident has not occurred anywhere and also the number of incidents with this condition is insignificant compared to the number of total incidents. The cases that have an address but no coordinates, or vice versa, would be maintained.

With the column of `participant_age1` we will fill the nulls with the average age of the participants, if this is not null.

Although many other columns have empty values, are columns that null values are possible, since they do not necessarily have to have values. Therefore, we will leave them as there are.

In [ ]:
print("- Number of Rows in the Original Dataset:", incidents_dataset.shape[0])

# Eliminate the data that have null in incident_characteristics1, adress, latitude and adress.
incidents_dataset = incidents_dataset.dropna(subset=['incident_characteristics1'], how='all')
incidents_dataset = incidents_dataset.dropna(subset=['latitude', 'longitude', 'address'], how='all')

# Rename the other nulls with no_data
valor_reemplazo = 'no_data'
actualGender = 'Male'
totalChilds = 0
totalTeens = 0
totalAdults = 0
totalMales = 0
totalFemales = 0
totalArrested = 0
totalUnharmed = 0 
def setAgeGroup(a):
    if a < 12:
        return 'Child 0-11'
    elif a < 18:
        return 'Teen 12-17'
    return 'Adult 18+'
def sumTotalNumber(a, t):
    if a is not None:
        return t
    else:
        return t + a

actualGender = 'Male'  # Asegúrate de inicializar actualGender antes del bucle si aún no está definido
totalChilds, totalTeens, totalAdults, totalMales, totalFemales, totalArrested, totalUnharmed = 0, 0, 0, 0, 0, 0, 0  # Inicializar las variables acumulativas

for index, x in incidents_dataset.iterrows():
    # Primero limpiamos las edades mínima, promedio y máxima de los participantes
    if pd.isnull(x['min_age_participants']):
        if not pd.isnull(x['avg_age_participants']):
            if not pd.isnull(x['max_age_participants']):
                k = x['avg_age_participants'] - (x['max_age_participants'] - x['avg_age_participants'])
                if k < 0:
                    incidents_dataset.at[index, 'min_age_participants'] = 0
                else:
                    incidents_dataset.at[index, 'min_age_participants'] = k
            else:
                incidents_dataset.at[index, 'min_age_participants'] = x['avg_age_participants']

    if pd.isnull(x['max_age_participants']):
        if not pd.isnull(x['avg_age_participants']):
            if not pd.isnull(x['min_age_participants']):
                incidents_dataset.at[index, 'max_age_participants'] = x['avg_age_participants'] + (x['avg_age_participants'] - x['min_age_participants'])
            else:
                incidents_dataset.at[index, 'max_age_participants'] = x['avg_age_participants']

    if pd.isnull(x['avg_age_participants']):
        if not (pd.isnull(x['min_age_participants']) or pd.isnull(x['max_age_participants'])):
            incidents_dataset.at[index, 'avg_age_participants'] = (float(x['min_age_participants']) + float(x['max_age_participants'])) / 2
        elif pd.isnull(x['min_age_participants']):
            incidents_dataset.at[index, 'avg_age_participants'] = float(x['max_age_participants'])
        else:
            incidents_dataset.at[index, 'avg_age_participants'] = float(x['min_age_participants'])

    # Para la edad aleatoria del participante, usaremos el promedio; si es nula, usamos la mínima o la máxima
    if pd.isnull(x['participant_age1']):
        if (not pd.isnull(x['avg_age_participants'])) and (x['avg_age_participants'] > 0):
            incidents_dataset.at[index, 'participant_age1'] = float(x['avg_age_participants'])
        elif (not pd.isnull(x['min_age_participants'])) and (x['min_age_participants'] > 0):
            incidents_dataset.at[index, 'participant_age1'] = float(x['min_age_participants'])
        elif (not pd.isnull(x['max_age_participants'])) and (x['max_age_participants'] > 0): 
            incidents_dataset.at[index, 'participant_age1'] = float(x['max_age_participants'])

    # Para el grupo de edad aleatorio, hacemos lo mismo que para participant_age1
    if pd.isnull(x['participant_age_group1']):
        if not pd.isnull(x['avg_age_participants']):
            incidents_dataset.at[index, 'participant_age_group1'] = setAgeGroup(float(x['avg_age_participants']))
        elif not pd.isnull(x['min_age_participants']):
            incidents_dataset.at[index, 'participant_age_group1'] = setAgeGroup(float(x['min_age_participants']))
        elif not pd.isnull(x['max_age_participants']):
            incidents_dataset.at[index, 'participant_age_group1'] = setAgeGroup(float(x['max_age_participants']))
        elif not pd.isnull(x['participant_age1']):
            incidents_dataset.at[index, 'participant_age_group1'] = setAgeGroup(float(x['participant_age1']))

    # Para la edad aleatoria del participante de cada fila, la llenamos con un género diferente al anterior
    if pd.isnull(x['participant_gender1']):
        incidents_dataset.at[index, 'participant_gender1'] = actualGender
        actualGender = 'Female' if actualGender == 'Male' else 'Male'

    # Acumular totales
    totalChilds = sumTotalNumber(x['n_participants_child'], totalChilds)
    totalTeens = sumTotalNumber(x['n_participants_teen'], totalTeens)
    totalAdults = sumTotalNumber(x['n_participants_adult'], totalAdults)
    totalMales = sumTotalNumber(x['n_males'], totalMales)
    totalFemales = sumTotalNumber(x['n_females'], totalFemales)
    totalArrested = sumTotalNumber(x['n_arrested'], totalArrested)
    totalUnharmed = sumTotalNumber(x['n_unharmed'], totalUnharmed)

#For the rest numeral attributes we just do the average of all the cases
incidents_dataset['n_participants_child'].fillna(round(totalChilds/len(incidents_dataset)), inplace=True)
incidents_dataset['n_participants_teen'].fillna(round(totalTeens/len(incidents_dataset)), inplace=True)
incidents_dataset['n_participants_adult'].fillna(round(totalAdults/len(incidents_dataset)), inplace=True)
incidents_dataset['n_males'].fillna(round(totalMales/len(incidents_dataset)), inplace=True)
incidents_dataset['n_females'].fillna(round(totalFemales/len(incidents_dataset)), inplace=True)
incidents_dataset['n_arrested'].fillna(round(totalArrested/len(incidents_dataset)), inplace=True)
incidents_dataset['n_unharmed'].fillna(round(totalUnharmed/len(incidents_dataset)), inplace=True)

# Print mising values label
missing_values = incidents_dataset.isnull().sum()
plt.figure(figsize=(12, 6))
sns.barplot(x=missing_values.index, y=missing_values.values, hue=missing_values.index, palette='flare', legend=False)
plt.xticks(rotation=45, ha='right')
plt.title('Sum of Missing Values in Incidents Dataset')
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.show()

After that, we will check for duplicated values in the datasets. We can see that the incidents dataset is the only one that has duplicates, so we remove them and then verify that we have done it correctly.

In [ ]:
# Check for duplicates
d1_duplicates = incidents_dataset.duplicated().sum()
print("- Number of Duplicates:", d1_duplicates)

# Remove duplicate rows
incidents_dataset.drop_duplicates(inplace=True)

# Verify the removal of duplicates
d1_duplicates_after_removal = incidents_dataset.duplicated().sum()
print("\n- Number of Duplicates After Removal:", d1_duplicates_after_removal)


In [ ]:
# Check for duplicates
d2_duplicates = pbsy_dataset.duplicated().sum()
print("- Number of Duplicates:", d2_duplicates)

In [ ]:
# Check for duplicates
d3_duplicates = ysdh_dataset.duplicated().sum()
print("- Number of Duplicates:", d3_duplicates)

After that, and before starting with further modifications, we will establish the same capitalization for the state column, because the elections dataset has the state in upper case, while the other two not. This will help with further merge and analysis.

In [ ]:

incidents_dataset['state'] = incidents_dataset['state'].str.upper()
pbsy_dataset['state'] = pbsy_dataset['state'].str.upper()
ysdh_dataset['state'] = ysdh_dataset['state'].str.upper()

### Distribution of variables

Then, we will see the distribution of the variables in the different datasest.

#### Poverty percentages evaluation

To start, we will see the distribution of the poverty rates by states.

In [ ]:
plt.figure(figsize=(15, 8))
sns.barplot(data=pbsy_dataset, x='state', y='povertyPercentage', hue='state', palette='viridis', dodge=False)
plt.title('Poverty Percentage by State')
plt.xlabel('State')
plt.ylabel('Poverty Percentage')
plt.xticks(rotation=45, ha='right')
plt.show()

In the previous chart, we can see that the first state is "United States". This is not correct, because the United States is the country, but not a state. Even tough we will not remove or modify it, we have to take it into account when doing analysis. In the cases that we analyze global poverty rates among the country, there is no problem using this dataset, but if we want to distinguish among states, or make analysis where the states have relevance, we should remove the rows that have the column state as "United States". After all the modifications we will create two different datasets for each case.

Then, we will make a boxplot so see the evolution of the poverty over the years, as well as the distribution of the poverty ranges in each year.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=pbsy_dataset, x='year', y='povertyPercentage', hue='year', palette='viridis', dodge=False)
plt.title('Poverty Percentage Over the Years')
plt.xlabel('Year')
plt.ylabel('Poverty Percentage')
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


We can see in the previous graph that there are no values in the year 2012. To check that we can filter the data by year and not *null* data, and check that there are no rows. Nevertheless, there are 52 rows with null values.

In [ ]:
entries_2012 = pbsy_dataset[pbsy_dataset['year'] == 2012]
entries_2012_null = pbsy_dataset[(pbsy_dataset['year'] == 2012) & (pbsy_dataset['povertyPercentage'].isnull())]

print('- Amount of entries for 2012: ' + str(entries_2012.shape[0]))
print('- Empty Amount of entries for 2012: ' + str(entries_2012_null.shape[0]))

So to fix this problem, we will use the *mean imputation* technique including some random noise, which consist on computing the mean of the observed values for each variable and imputing the missing values for that variable by this mean, modifying it slightly for each case.

In this case, instead of getting the complete average, we impute the missing values using the average of the adjacent years (2011 and 2013).

In [ ]:
import numpy as np

# Value to control the strength of the noise
noise_strength = 0.05

# Set the entries of 2012 as missing rows
missing_rows  = entries_2012_null

# Get the average of 'povertyPercentage' for the adjacent years
average_2011 = pbsy_dataset.loc[pbsy_dataset['year'] == 2011].groupby('state')['povertyPercentage'].transform('mean')
average_2013 = pbsy_dataset.loc[pbsy_dataset['year'] == 2013].groupby('state')['povertyPercentage'].transform('mean')

# If there are missing rows, calculate the average with added random noise
if not missing_rows.empty:
    # Calculate the average with added random noise
    noisy_average = np.random.uniform((1 - noise_strength) * average_2011, (1 + noise_strength) * average_2013)

    # Fill the missing values with the calculated averages with noise
    pbsy_dataset.loc[missing_rows.index, 'povertyPercentage'] = noisy_average


# Check if there are any missing values remaining
entries_2012_null = pbsy_dataset[(pbsy_dataset['year'] == 2012) & (pbsy_dataset['povertyPercentage'].isnull())]
print('- Empty Amount of entries for 2012: ' + str(entries_2012_null.shape[0]) + '\n')

#Print some rows as example
entries_2012 = pbsy_dataset[pbsy_dataset['year'] == 2012]
print(entries_2012.head())


Now if we repeat the previous graph we can see that there are no missing values in 2012.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=pbsy_dataset, x='year', y='povertyPercentage', hue='year', palette='viridis', dodge=False)
plt.title('Poverty Percentage Over the Years')
plt.xlabel('Year')
plt.ylabel('Poverty Percentage')
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

To conclude, with all the modifications done, we will see the general distribution of the poverty rates throughout all the US. We can see that even tough there are some rates that are higher, the distribution is skewed right.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=pbsy_dataset, x='povertyPercentage', bins=20, kde=True, color='skyblue')
plt.title('Distribution of Poverty Percentage')
plt.xlabel('Poverty Percentage')
plt.ylabel('Frequency')
plt.show()


After all the modifications we will create the two datasets with and without the "United States" as state.

In [ ]:
# Filter rows with 'United States' as state in pbsy_dataset
pbsy_dataset_only_states = pbsy_dataset[pbsy_dataset['state'] != 'United States']

#### Evaluation of elections dataset

Similar analysis will be performed for the elections dataset. Firstly, we will check the evolution of the amount of votes per year. In general, we can say that the participation in the elections has been increasing by the years.

Nevertheless, we know that the elections are every 4 years, while in this dataset there are entries every 2 years. That's why in the graph we can see the difference in the amount of votes every two years. We assume that this votes correspond to the midterm that the US does for the elections. For now we will keep it until concluding with the modifications. After finishing with the modifications we will separate in two datasets: one with all the voting campaigns and the other one just with the elections.

In [ ]:
yearly_total_votes = ysdh_dataset.groupby('year')['totalvotes'].sum().reset_index(name='total_votes')

plt.figure(figsize=(12, 6))
sns.barplot(data=yearly_total_votes, x='year', y='total_votes', hue='year', palette='flare')
plt.title('Total Votes by Year')
plt.xlabel('Year')
plt.ylabel('Total Votes')
plt.show()


With the scatter plot of the relations between candidate votes and total votes, we can detect some anomalies in the dataset. We have many values in the (0,0), which means that `candidatevotes` and `totalvotes` are the same and also we can see points in the top right of the chart with huge values, which are clearly anomalies as well.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='candidatevotes', y='totalvotes', data=ysdh_dataset)
plt.title('Scatter Plot of Candidate Votes vs Total Votes')
plt.xlabel('Candidate Votes')
plt.ylabel('Total Votes')
plt.show()


To fix that we calculate the Interquartile Range (IQR) for both `candidatevotes` and `totalvotes`, a measure of statistical dispersion based on quartiles, which is used to define upper and lower bounds for identifying outliers. Quartiles divide a dataset into four equal parts. The first quartile (Q1) represents the 25th percentile, and the third quartile (Q3) represents the 75th percentile. The IQR is the range between Q1 and Q3. Outliers were identified by defining bounds outside 1.5 times the IQR from the quartiles. This method is effective for detecting data points that deviate significantly from the central tendency of the dataset.

Those entires that deviate from the tendency have been removed from the main dataset, to ensure that our subsequent analyses and visualizations are not influenced by extreme values.

In [ ]:
# Calculate the IQR for 'candidatevotes' and 'totalvotes'
Q1_candidate = ysdh_dataset['candidatevotes'].quantile(0.25)
Q3_candidate = ysdh_dataset['candidatevotes'].quantile(0.75)
IQR_candidate = Q3_candidate - Q1_candidate

Q1_total = ysdh_dataset['totalvotes'].quantile(0.25)
Q3_total = ysdh_dataset['totalvotes'].quantile(0.75)
IQR_total = Q3_total - Q1_total

# Define the upper and lower bounds for outliers
lower_bound_candidate = Q1_candidate - 1.5 * IQR_candidate
upper_bound_candidate = Q3_candidate + 1.5 * IQR_candidate

lower_bound_total = Q1_total - 1.5 * IQR_total
upper_bound_total = Q3_total + 1.5 * IQR_total

# Identify outliers
outliers = ((ysdh_dataset['candidatevotes'] < lower_bound_candidate) | 
            (ysdh_dataset['candidatevotes'] > upper_bound_candidate) |
            (ysdh_dataset['totalvotes'] < lower_bound_total) |
            (ysdh_dataset['totalvotes'] > upper_bound_total))

# Print the number of outliers
print("In the dataset there are " + str(ysdh_dataset[outliers].shape[0]) + " outliers")
print(ysdh_dataset[outliers].head())

# Remove outliers
ysdh_dataset = ysdh_dataset[~outliers]


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='candidatevotes', y='totalvotes', data=ysdh_dataset)
plt.title('Scatter Plot of Candidate Votes vs Total Votes')
plt.xlabel('Candidate Votes')
plt.ylabel('Total Votes')
plt.show()

Apart from that the big problem that we have with this dataset is its structure. The elections results are divided by states, which it is correct, but also by congressional districts, which is not relevant for our analysis.

Moreover, we do not have the information about the amount of votes that each party obtained in each congressional district. We only know which party was the winner in each congressional party, and how many votes they got. Then we have the total amount of votes, but we do not know how are distributed among the rest of the parties.

As an example, we will get the first year and the first state (1976 and Alabama):

In [ ]:
rows_1976_alabama = ysdh_dataset[(ysdh_dataset['year'] == 1976) & (ysdh_dataset['state'] == 'ALABAMA')]
print(rows_1976_alabama)


In [ ]:
alabama_1976_d1 = rows_1976_alabama.iloc[0]

print('- Votes for Republicans in Alabama district 1: ' + str(alabama_1976_d1['candidatevotes']))
print('- Votes for other parties in Alabama district 1: ' + str(alabama_1976_d1['totalvotes'] - alabama_1976_d1['candidatevotes']))

Since in the US mainly all the votes goes for republicans or democrats, it could be said that the other votes, could be set to the other party in each row. However, we check that the dataset has more parties apart from republicans and democrats, so this modification would not be entirely correct.

In [ ]:
unique_parties = ysdh_dataset['party'].unique()
print("Different Parties:", unique_parties)


Anyway, we checked which is the percentage that these little parties occupy in the results. The result was less than a 1% of the entire votes during the history.

In [ ]:
total_votes_by_party = ysdh_dataset.groupby('party')['totalvotes'].sum().reset_index()

# Calculate the percentage of total votes for each party
total_votes_by_party['percentage'] = (total_votes_by_party['totalvotes'] / total_votes_by_party['totalvotes'].sum()) * 100

print(total_votes_by_party)

# Filter out Republicans and Democrats
other_parties = total_votes_by_party[~total_votes_by_party['party'].isin(['REPUBLICAN', 'DEMOCRAT'])]

# Calculate the sum of the percentage for other parties
sum_percentage_other_parties = other_parties['percentage'].sum()

print(f"\nSum of the percentage for other parties: {sum_percentage_other_parties:.2f}%")



Therefore, we decide to remove these parties and just have the republicans and democrats.

- *FOGLIETTA (DEMOCRAT)* will convert in *DEMOCRAT*
- *DEMOCRATIC-FARMER-LABOR* will convert in DEMOCRAT
- *INDEPENDENT* will convert randomly in *DEMOCRAT* or *REPUBLICAN*
- *INDEPENDENT-REPUBLICAN* will convert in *REPUBLICAN*

In [ ]:
import numpy as np

# Update 'party' column
ysdh_dataset['party'] = np.where(ysdh_dataset['party'] == 'FOGLIETTA (DEMOCRAT)', 'DEMOCRAT', ysdh_dataset['party'])
ysdh_dataset['party'] = np.where(ysdh_dataset['party'] == 'DEMOCRATIC-FARMER-LABOR', 'DEMOCRAT', ysdh_dataset['party'])
ysdh_dataset['party'] = np.where(ysdh_dataset['party'] == 'INDEPENDENT-REPUBLICAN', 'REPUBLICAN', ysdh_dataset['party'])

# Randomly assign 'DEMOCRAT' or 'REPUBLICAN' for 'INDEPENDENT'
independent_indices = ysdh_dataset[ysdh_dataset['party'] == 'INDEPENDENT'].index
ysdh_dataset.loc[independent_indices, 'party'] = np.random.choice(['DEMOCRAT', 'REPUBLICAN'], size=len(independent_indices))

# Verify the changes
print(ysdh_dataset['party'].unique())


After having just the two main parties, the first modification of the dataset will be to include two new columns: `republican_votes` and `democrat_votes`.

If the party is republican, we know that `candidatevotes` are republican votes, so the rest are democrats votes (`totalvotes` - `candidatevotes`), and the same in the opposite way. In order to take into account the remaining votes for other political parties, we have reduced the calculation of the remaining votes to 1% (which is the average obtained previously, 0.87%).

In [ ]:
def calculate_votes(row):
    resting_coef = 0.01
    if row['party'] == 'REPUBLICAN':
        return pd.Series([row['candidatevotes'], round((row['totalvotes'] - row['candidatevotes']) * (1 - resting_coef))])
    else:
        return pd.Series([round((row['totalvotes'] - row['candidatevotes']) * (1 - resting_coef)), row['candidatevotes']])

# Apply the custom function to create new columns
ysdh_dataset[['republican_votes', 'democrat_votes']] = ysdh_dataset.apply(calculate_votes, axis=1)

# Print the updated dataset
print(ysdh_dataset.head())


After this modification we can remove the `candidatevotes` columns, because it is useless for us.

In [ ]:
# Drop the 'candidatevotes' column
ysdh_dataset.drop('candidatevotes', axis=1, inplace=True)

# Print the updated dataset
print(ysdh_dataset[0:10])

To conclude with the modifications, we will remove the congressional_district information and merge the rows that have the same year and state. This way, we will have the information about the votes that each party has obtained in each year and state, which has been the winner, and the total amount of votes pero party and in total.

In [ ]:
# Group by 'year' and 'state', summing up the votes
voting_grouped_data = ysdh_dataset.groupby(['year', 'state']).agg({
    'totalvotes': 'sum',
    'republican_votes': 'sum',
    'democrat_votes': 'sum'
}).reset_index()

# Update the 'party' column based on the comparison of votes
voting_grouped_data['party'] = voting_grouped_data.apply(lambda row: 'REPUBLICAN' if row['republican_votes'] > row['democrat_votes'] else 'DEMOCRAT', axis=1)

# Print the updated dataset
print(voting_grouped_data.head())

Now that me conclude with the fixing, we will separate the dataset in two: one with all the voting campaigns and the other one just with the elections.

In [ ]:
midterm_dataset = voting_grouped_data[voting_grouped_data['year'] % 4 != 0]
elections_dataset = voting_grouped_data[voting_grouped_data['year'] % 4 == 0]

# Display or print the first few rows to verify
print("Mid term voting dataset:")
print(midterm_dataset['year'].unique())

print("\nElections dataset:")
print(elections_dataset['year'].unique())


In [ ]:
# Plot the total votes for each party per year
plt.figure(figsize=(12, 6))
sns.barplot(data=elections_dataset, x='year', y='totalvotes', hue='party', palette={'DEMOCRAT': 'blue', 'REPUBLICAN': 'red'}, errorbar=None)
plt.title('Elections results Per Year')
plt.xlabel('Year')
plt.ylabel('Total Votes')
plt.show()


#### Evolution of incidents over time

The evolution of the incidents over time is one of the most relevant topics to analyze, so in the next graph we display it. We can see something strange. There are values that start around 2014, that finish around middle of 2018, and then values that restart in 2028. We don't know why that data is in the dataset, if by mistake or if it is some kind of prediction that they have made, where they have the incident data that they would like to have for the future. Whatever it is, for our analysis the future data is not relevant, so we are going to remove it.

In [ ]:
# Count the number of incidents for each date
incident_counts = incidents_dataset['date'].value_counts().sort_index()

# Plotting the evolution of the amount of incidents over time
plt.figure(figsize=(12, 6))
sns.lineplot(x=incident_counts.index, y=incident_counts.values, marker='o', color='skyblue')
plt.title('Evolution of Incidents Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Filter out rows with 'date' in the future
incidents_dataset = incidents_dataset[incidents_dataset['date'].dt.year < 2028]

# Verify the changes
print(incidents_dataset['date'].describe())


In [ ]:
# Count the number of incidents for each date
incident_counts = incidents_dataset['date'].value_counts().sort_index()

# Plotting the evolution of the amount of incidents over time
plt.figure(figsize=(12, 6))
sns.lineplot(x=incident_counts.index, y=incident_counts.values, marker='o', color='skyblue')
plt.title('Evolution of Incidents Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.show()

Next issue that we can see is the difference between the incidents until 2014, with the incidents between 2014 and 2018. This does not occur because 2013 was the safest year of the history, but because there are way less values in the dataset than in the rest of the years. Therefore, in order not to alter the results, we have decided to remove entries older than 2014.

In [ ]:
# Extract the year from the 'date' column and create a new 'year' column
incidents_dataset['year'] = incidents_dataset['date'].dt.year

# Count the number of incidents by year
incident_counts_by_year = incidents_dataset['year'].value_counts().sort_index()

# Display the counts
print("Number of Incidents by Year:")
print(incident_counts_by_year)

In [ ]:
# Filter out rows with 'date' in the future
incidents_dataset = incidents_dataset[incidents_dataset['date'].dt.year > 2013]

# Verify the changes
print(incidents_dataset['date'].describe())

Final result of the plot would be the next one:

In [ ]:
# Count the number of incidents for each date
incident_counts = incidents_dataset['date'].value_counts().sort_index()

# Plotting the evolution of the amount of incidents over time
plt.figure(figsize=(12, 6))
sns.lineplot(x=incident_counts.index, y=incident_counts.values, marker='o', color='skyblue')
plt.title('Evolution of Incidents Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.show()

#### Geographical distribution of incidents

In [ ]:
import geopandas as gpd

world_filepath = './110m_cultural/ne_110m_admin_0_countries.shp'
world = gpd.read_file(world_filepath)

# Create a GeoDataFrame from the incidents_dataset DataFrame
gdf = gpd.GeoDataFrame(incidents_dataset, geometry=gpd.points_from_xy(incidents_dataset.longitude, incidents_dataset.latitude))

# Plot the world map
world.plot(figsize=(12, 8), color='lightgrey', edgecolor='black')

# Plot the scatter plot on top of the world map
scatter_plot = sns.scatterplot(x='longitude', y='latitude', data=incidents_dataset, hue='n_killed', palette='viridis',
                               size='n_injured', sizes=(20, 200), alpha=0.7)

# Move the legend outside the chart
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.title('Geographical Distribution of Incidents')
plt.show()

##### Geographical data fixing

As we see in the data understanding section, there were some outliers in the geographical data, since it was visible that there were some incidents around India, when there would be just incidents from the USA.

To fix that we refined the filtering criteria based on the latitude and longitude values. We set the latitude and longitude ranges to cover the area of the United States, including Alaska and Hawaii that have different coordinates that the main US region. These adjusted ranges ensure that incidents falling within the geographical coordinates of the entire United States are retained in the filtered dataset. This refined filtering approach allows for the accurate representation of incident locations on the geographical scatter plot while eliminating data points located outside the intended area of interest.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming the latitude and longitude ranges for the United States
us_latitude_range = (18, 71)
us_longitude_range = (-179, -66)

# Filter out incidents outside the US coordinates
incidents_dataset_us = incidents_dataset[(incidents_dataset['latitude'].between(*us_latitude_range)) &
                        (incidents_dataset['longitude'].between(*us_longitude_range))]

# Create a GeoDataFrame from the filtered dataset
gdf_us = gpd.GeoDataFrame(incidents_dataset_us, geometry=gpd.points_from_xy(incidents_dataset_us.longitude, incidents_dataset_us.latitude))

# Plot the world map
world_filepath = './110m_cultural/ne_110m_admin_0_countries.shp'
world.plot(figsize=(12, 8), color='lightgrey', edgecolor='black')

# Plot the scatter plot on top of the world map for the US incidents
scatter_plot = sns.scatterplot(x='longitude', y='latitude', data=incidents_dataset_us, hue='n_killed', palette='viridis',
                               size='n_injured', sizes=(20, 200), alpha=0.7)

# Move the legend outside the chart
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.title('Geographical Distribution of Incidents in the US')
plt.show()


#### Distribution of participant age

Knowing the characteristics of the participants in the incidents it is one of the keys of our analysis, so we will start with the age of the participants. In the next graph we can have a preview of the distribution of the ages. We can see that there are ages that are greater than 100 years, including an entry with 311 years which is the maximum. This is almost impossible, so we will remove the values that have participants older than 110 years.

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=incidents_dataset, x='participant_age1', bins=20, kde=True, color='skyblue')
plt.title('Distribution of Participant Ages')
plt.show()

In [ ]:
# Remove participants older than 110 years
incidents_dataset = incidents_dataset[incidents_dataset['participant_age1'] <= 110]

# Verify the changes
print(incidents_dataset['participant_age1'].describe())


In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=incidents_dataset, x='participant_age1', bins=20, kde=True, color='skyblue')
plt.title('Distribution of Participant Ages')
plt.show()

#### Distribution of participant gender

Then we will analyze the gender of the participants. There are male and female values, but we also have an entry with the gender 'Male, female'. Since it is just one entry, and to simplify the analysis, we will remove this entry, converting in to "Male" and just have female and male categories.

In [ ]:
gender_counts = incidents_dataset['participant_gender1'].value_counts()

# Print the counts
print("Gender Distribution:")
print(gender_counts)

In [ ]:
# Remove rows with 'participant_gender1' equal to "Male, female"
# Replace the column values where 'participant_gender1' is "Male, female" with "Male"
incidents_dataset['participant_gender1'] = incidents_dataset['participant_gender1'].replace("Male, female", "Male")


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=incidents_dataset, x='participant_gender1', palette='flare', hue='participant_gender1', legend=False)
plt.title('Distribution of Participant Genders')
plt.show()

#### Number of Participants and Casualties

In the next graph we show the distribution of the number of participants, and the number of injured, killed, arrested and unharmed participants. Even tough there are some outliers, the distribution is skewed to the right, since in most cases the number of participants is low and, on the contrary, there are very few cases with a large number of participants.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=incidents_dataset[['n_participants', 'n_killed', 'n_injured','n_arrested', 'n_unharmed']])
plt.title('Number of Participants and Casualties')
plt.show()

#### Incident Characteristics

In the next graph the top 10 most occurred incidents will be displayed, to see which where the main ones.

In [ ]:
# Get the total number of different incident characteristics and print them
total_characteristics = incidents_dataset['incident_characteristics1'].nunique()
print('- Number of different incident characteristics: ' + str(total_characteristics))

# Get the top 10 most common incident characteristics
top10_characteristics = incidents_dataset['incident_characteristics1'].value_counts().nlargest(10).index

# Create a countplot with the top 10 incident characteristics
plt.figure(figsize=(10, 6))
sns.countplot(data=incidents_dataset, y='incident_characteristics1', order=top10_characteristics, palette='muted', hue='incident_characteristics1', legend=False)
plt.title('Top 10 Most Common Incident Characteristics')
plt.show()


### Pairwise correlation

To perform pairwise correlation analysis, `corr()` function of `pandas` can be used, to compute the correlation matrix and then visualize it using a heatmap.

The goal of pairwise correlation analysis is to understand the linear relationship between pairs of variables. The correlation coefficient ranges from -1 to 1, where:

- 1 indicates a perfect positive correlation.
- -1 indicates a perfect negative correlation.
- 0 indicates no correlation.

In the pairwise correlation matrix of the incidents dataset we do not have many interesting correlations to consider. We can observe a strong positive correlation with all the variables that measure the age, but that is obvious so it is not considerable. The most interesting fact could be the correlation between the incidents consequences and the gender. There is a stronger correlation between the consequences (`n_killed`, `n_arrested`, `n_injured`...) and  males than females.

In [ ]:
# Select numerical columns for correlation analysis
numerical_columns = incidents_dataset.select_dtypes(include=['number'])

# Compute the correlation matrix
correlation_matrix = numerical_columns.corr()

# Plot a heatmap of the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Pairwise Correlation Matrix')
plt.show()

#### Incidents and elections correlation:

Checking the correlation matrix between the incidents and the elections, we can not say that there is any correlation to consider, since all the values are around 0.

In [ ]:
# Define columns of interest in incidents dataset
incidents_columns_of_interest = [
    'participant_age1', 'avg_age_participants', 'n_males', 'n_females', 
    'n_participants_child', 'n_participants_teen', 'n_killed', 'n_injured', 'n_arrested'
]

# Merge datasets on common keys
merged_dataset = pd.merge(
    incidents_dataset[incidents_columns_of_interest],
    ysdh_dataset,
    left_on=[incidents_dataset['state'], incidents_dataset['date'].dt.year],
    right_on=['state', 'year']
)

# Select numerical columns for correlation analysis
numerical_columns_merged = merged_dataset.select_dtypes(include=['number'])

# Compute the correlation matrix
correlation_matrix_merged = numerical_columns_merged.corr()

# Plot a heatmap of the correlation matrix
plt.figure(figsize=(11, 7))
sns.heatmap(correlation_matrix_merged, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
plt.show()


#### Incidents and poverty rates correlation

In the case of the incidents and poverty rates, there is no clear correlation either.

In [ ]:
# Merge datasets on common keys
merged_dataset = pd.merge(
    incidents_dataset[incidents_columns_of_interest],
    pbsy_dataset_only_states,
    left_on=[incidents_dataset['state'], incidents_dataset['date'].dt.year],
    right_on=['state', 'year']
)

# Select numerical columns for correlation analysis
numerical_columns_merged = merged_dataset.select_dtypes(include=['number'])

# Compute the correlation matrix
correlation_matrix_merged = numerical_columns_merged.corr()

# Plot a heatmap of the correlation matrix
plt.figure(figsize=(11, 7))
sns.heatmap(correlation_matrix_merged, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

#### Poverty rates and elections correlation

In the case of the correlation between poverty rates and elections, even tough there is no a really clear and obvious correlation, we can observe an interesting difference between the correlation of the poverty percentage and the political party votes.

The poverty percentage has a negative correlation of -0.34 with democrat votes, while the correlation with republican votes is almost zero, -0.06.

In [ ]:
# Merge datasets on common keys
merged_dataset = pd.merge(
    ysdh_dataset,
    pbsy_dataset_only_states,
    left_on=['state', 'year'],
    right_on=['state', 'year']
)

# Select numerical columns for correlation analysis
numerical_columns_merged = merged_dataset.select_dtypes(include=['number'])

# Compute the correlation matrix
correlation_matrix_merged = numerical_columns_merged.corr()

# Plot a heatmap of the correlation matrix
plt.figure(figsize=(11, 7))
sns.heatmap(correlation_matrix_merged, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

### New feature extraction

After making many modifications to the datasets, to conclude the data preparation aspect, we are going to improve the quality of the data, increasing the knowledge about it in order to extract new interesting features to describe the incidents. To do so, we are going to obtain several indicators for an incident by evaluating multiple aspects.

The first question that has been addressed is the following:

*How many males are involved in the incident w.r.t. the total number of males involved in incidents for the same city and in the same period?*

In this case the period that we will analyze will be a year, so first, we add a new column representing the year extracted from the 'date' column. Subsequently, we group the dataset by city and year, calculating the total number of males involved in incidents for each combination. The total male information is then merged back into the original dataset (`n_males_total` column). To ensure accurate proportions in the subsequent calculations, any NaN or zero values in the total male column are replaced with 1. The function proceeds to compute the proportion of males involved in each incident (`proportion_males`), considering the total numbers.

In [ ]:
# Create a new column for the year
incidents_dataset['year'] = incidents_dataset['date'].dt.year

# Group by city and year, calculate total males involved in incidents
total_males_per_city_year = incidents_dataset.groupby(['city_or_county', 'year'])['n_males'].sum().reset_index()

# Merge total males information back into the original dataset
incidents_with_total_males = pd.merge(
    incidents_dataset,
    total_males_per_city_year,
    on=['city_or_county', 'year'],
    suffixes=('', '_total')
)

# Replace NaN or zero values in 'n_males_total' with 1 to avoid division issues
incidents_with_total_males['n_males_total'].replace({0: 1}, inplace=True)

# Calculate the proportion of males involved in each incident
incidents_with_total_males['proportion_males'] = (
    incidents_with_total_males['n_males'] / incidents_with_total_males['n_males_total']
)

# Display the result (you can adjust the columns displayed as needed)
result_columns = ['date', 'city_or_county', 'n_males', 'n_males_total', 'proportion_males', 'incident_characteristics1']
result = incidents_with_total_males[result_columns].sort_values(by=['city_or_county', 'date'])

# Display the result to check if the proportions are correct
print(result.head())


With the previous modified dataset we can see the information of the proportion of males in incidents in a specific city. For testing we set manually the name of the city, but the user can modify it including the name of the desired city.

In [ ]:
# Sort data by date
incidents_with_total_males = incidents_with_total_males.sort_values(by='date')

# Plot the proportion of males over time for a specific city
# (For testing we specify manually the name of the city, but in production, user will be able to specify the city that he wants)
city_to_plot = 'Boise'
#city_to_plot = input('Introduce the name of a city: ')

# Filter data for the selected city
city_data = incidents_with_total_males[incidents_with_total_males['city_or_county'] == city_to_plot]

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(city_data['date'], city_data['proportion_males'] * 100, linestyle='-', color='g')
plt.title(f'Proportion of Males Involved in Incidents Over Time - {city_to_plot}')
plt.xlabel('Date')
plt.ylabel('Proportion of Males')
plt.grid(True)
plt.show()



Next question was the next one:

*How many injured and killed people have been involved w.r.t the total injured and killed people in the same congressional district in a given period of time?*

The approach to achieve the result of this question has been similar as the previous one. To start, we have grouped the incidents dataset based on the congressional district and year, calculating the total number of individuals injured and killed in incidents for each combination. This information is then merged back into the original dataset. As it was done in the previous case, any NaN or zero values in the total injured and killed columns are replaced with 1. The function proceeds to compute the proportion of injured and killed people involved in each incident, considering the total numbers. With this data, the new dataset has 4 more columns: `n_injured_total`, `n_killed_total`, `proportion_injured` and `proportion_killed`. Finally, the dataset is filtered for a specific congressional district (specified as 1 for testing, but it can be set by the user), to analyze more detailed information.

In [ ]:
# Group by congressional district, year, and calculate total injured and killed people
total_injured_killed_per_district_year = incidents_dataset.groupby(['congressional_district', 'year'])[['n_injured', 'n_killed']].sum().reset_index()

# Merge total injured and killed information back into the original dataset
incidents_with_total_injured_killed = pd.merge(
    incidents_dataset,
    total_injured_killed_per_district_year,
    on=['congressional_district', 'year'],
    suffixes=('', '_total')
)

# Replace NaN or zero values in 'n_injured_total' and 'n_killed_total' with 1 to avoid division issues
incidents_with_total_injured_killed['n_injured_total'].replace({0: 1}, inplace=True)
incidents_with_total_injured_killed['n_killed_total'].replace({0: 1}, inplace=True)

# Calculate the proportion of injured and killed people involved in each incident
incidents_with_total_injured_killed['proportion_injured'] = (
    incidents_with_total_injured_killed['n_injured'] / incidents_with_total_injured_killed['n_injured_total']
)
incidents_with_total_injured_killed['proportion_killed'] = (
    incidents_with_total_injured_killed['n_killed'] / incidents_with_total_injured_killed['n_killed_total']
)

# Filter data for a specific congressional district
# (For testing we specify manually the number of the congressional district, but in production, user will be able to specify the number that he wants)
congressional_district_to_plot = 1
#congressional_district_to_plot = input('Introduce the number of the congressional district: ')


# Filter data for the selected congressional district
incidents_with_total_injured_killed_in_district = incidents_with_total_injured_killed[incidents_with_total_injured_killed['congressional_district'] == congressional_district_to_plot]

print(incidents_with_total_injured_killed_in_district.head())


The third part consists on the addition of new columns regarding to the ratios of participant and participants consequences in the incidents: killed, injured, arrested and unharmed people.

To obtain this dataset, we make the calculation of the ratios using the required columns and inserting them in new columns. After including the columns, we have reduced the new dataset columns, to just the ones that we are interested in.

In [ ]:
# Calculate the ratio of killed people to participants in each incident
incidents_with_participant_ratios = incidents_dataset.copy()
incidents_with_participant_ratios['kill_participant_ratio'] = incidents_dataset['n_killed'] / incidents_dataset['n_participants']
incidents_with_participant_ratios['injured_participant_ratio'] = incidents_dataset['n_injured'] / incidents_dataset['n_participants']
incidents_with_participant_ratios['arrested_participant_ratio'] = incidents_dataset['n_arrested'] / incidents_dataset['n_participants']
incidents_with_participant_ratios['unharmed_participant_ratio'] = incidents_dataset['n_unharmed'] / incidents_dataset['n_participants']

# Display the result (you can adjust the columns displayed as needed)
result_columns = ['date', 'state', 'city_or_county', 'n_killed', 'kill_participant_ratio',
                  'n_injured', 'injured_participant_ratio', 
                  'n_arrested', 'arrested_participant_ratio',
                  'n_unharmed', 'unharmed_participant_ratio',
                  'n_participants', 'incident_characteristics1']
incidents_with_participant_ratios = incidents_with_participant_ratios[result_columns].sort_values(by='date')

# Print the result
print(incidents_with_participant_ratios.head())


As an example of visualization of the previous dataset, we have created the next plot, which shows the average ratios of the incident consequences for a specific city (As it done before, it is set manually for testing, but it can be customized).

In [ ]:
# Choose a specific city for filtering
selected_city = 'Boise'
#selected_city = input('Introduce the name of a city: ')

# Filter data for the selected city
city_data = incidents_with_participant_ratios[incidents_with_participant_ratios['city_or_county'] == selected_city]

# Group by year and calculate the average ratios
average_ratios_per_year = city_data.groupby(city_data['date'].dt.year)[
    ['kill_participant_ratio', 'injured_participant_ratio', 'arrested_participant_ratio', 'unharmed_participant_ratio']
].mean().reset_index()

# Plotting
plt.figure(figsize=(9, 5))
bar_width = 0.2
bar_locations = range(len(average_ratios_per_year['date']))

plt.bar(bar_locations, average_ratios_per_year['kill_participant_ratio'], width=bar_width, label='Kill Participant Ratio', color='r')
plt.bar([i + bar_width for i in bar_locations], average_ratios_per_year['injured_participant_ratio'], width=bar_width, label='Injured Participant Ratio', color='b')
plt.bar([i + 2 * bar_width for i in bar_locations], average_ratios_per_year['arrested_participant_ratio'], width=bar_width, label='Arrested Participant Ratio', color='g')
plt.bar([i + 3 * bar_width for i in bar_locations], average_ratios_per_year['unharmed_participant_ratio'], width=bar_width, label='Unharmed Participant Ratio', color='purple')

plt.title(f'Yearly Average Participant Ratios - {selected_city}')
plt.xlabel('Year')
plt.ylabel('Average Ratio')
plt.xticks([i + 1.5 * bar_width for i in bar_locations], average_ratios_per_year['date'])
plt.legend()
plt.show()



To conclude with the exploration of new features we have calculated the average age of participants in incidents for each city or state to then examine how it changes over different periods.

After creating the dataset with the average age participants of the dataset grouped by city and year, we display the results with three cities to compare. This way, in the plot we can see the difference between the evolution of participants average age over the different cities.

In [ ]:
# Create a new column for the year
incidents_dataset['year'] = incidents_dataset['date'].dt.year

# Group by city, state, and year, calculate the average age of participants
average_age_per_city_year = incidents_dataset.groupby(['city_or_county', 'year'])['avg_age_participants'].mean().reset_index()

# List of cities to compare
cities_to_compare = ['San Francisco', 'Las Vegas', 'Orlando']

# Filter data for the selected cities
average_age_per_city_year = average_age_per_city_year[average_age_per_city_year['city_or_county'].isin(cities_to_compare)]

# Plotting the average age over different periods for selected cities
plt.figure(figsize=(12, 4))
sns.lineplot(x='year', y='avg_age_participants', hue='city_or_county', data=average_age_per_city_year, marker='o')
plt.title('Average Age of Participants in Incidents')
plt.xlabel('Year')
plt.ylabel('Average Age')
plt.legend(title='City or County', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.show()


In order to improve the new features obtained in the previous exploration, we add some modifications to the previous approach. Here the function groups the dataset by incident characteristic, city and year, and then calculates the average age of the participants. This dataset is filtered by desired incident types and cities. Then the result is shown in a graph where we can observe the differences in average ages of the different cities and incident types.

In [ ]:
# Group by incident characteristics, city, and year, and calculate the average age of participants
average_age_per_incident_type_city_year = incidents_dataset.groupby(['incident_characteristics1', 'city_or_county', 'year'])['avg_age_participants'].mean().reset_index()

# List of incident types to compare
incident_types_to_compare = ['Shot - Wounded/Injured', 'Non-Shooting Incident']

# List of cities to compare
cities_to_compare = ['San Francisco', 'Las Vegas', 'Orlando']

# Filter data for the selected incident types and cities
average_age_per_incident_type_city_year = average_age_per_incident_type_city_year[
    (average_age_per_incident_type_city_year['incident_characteristics1'].isin(incident_types_to_compare)) &
    (average_age_per_incident_type_city_year['city_or_county'].isin(cities_to_compare))
]

# Plotting the average age over different periods for selected incident types and cities
plt.figure(figsize=(12, 6))
sns.lineplot(x='year', y='avg_age_participants', hue='city_or_county', 
             style='incident_characteristics1', data=average_age_per_incident_type_city_year, marker='o', errorbar=None)
plt.title('Average Age of Participants by Types and Cities')
plt.xlabel('Year')
plt.ylabel('Average Age')
plt.legend(title='Incident Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()